**About Book Crossing Dataset**<br>

This dataset has been compiled by Cai-Nicolas Ziegler in 2004, and it comprises of three tables for users, books and ratings. Explicit ratings are expressed on a scale from 1-10 (higher values denoting higher appreciation) and implicit rating is expressed by 0.

Reference: http://www2.informatik.uni-freiburg.de/~cziegler/BX/ 

**Objective**

This project entails building a Book Recommender System for users based on user-based and item-based collaborative filtering approaches.

In [0]:
! pip install surprise

In [65]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Execute the below cell to load the datasets

In [0]:
books = pd.read_csv("/content/drive/My Drive/AI_ML_CLass/Assignments/external/books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']

In [0]:
users = pd.read_csv('/content/drive/My Drive/AI_ML_CLass/Assignments/external/users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']

In [0]:
ratings = pd.read_csv('/content/drive/My Drive/AI_ML_CLass/Assignments/external/ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

### Check no.of records and features given in each dataset

In [10]:
books.shape

(271360, 8)

In [11]:
users.shape

(278858, 3)

In [12]:
ratings.shape

(1149780, 3)

## Exploring books dataset

In [13]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### Drop last three columns containing image URLs which will not be required for analysis

In [0]:
books=books.drop(columns=['imageUrlS','imageUrlM','imageUrlL'])

In [15]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


**yearOfPublication**

### Check unique values of yearOfPublication


In [18]:
books.yearOfPublication.unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

As it can be seen from above that there are some incorrect entries in this field. It looks like Publisher names 'DK Publishing Inc' and 'Gallimard' have been incorrectly loaded as yearOfPublication in dataset due to some errors in csv file.


Also some of the entries are strings and same years have been entered as numbers in some places. We will try to fix these things in the coming questions.

### Check the rows having 'DK Publishing Inc' as yearOfPublication

In [24]:
books[books.yearOfPublication=='DK Publishing Inc']

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


### Drop the rows having `'DK Publishing Inc'` and `'Gallimard'` as `yearOfPublication`

In [0]:
books=books.drop(index=books[books.yearOfPublication=='DK Publishing Inc'].index,axis=1) # books[books.yearOfPublication=='DK Publishing Inc']

In [0]:
books=books.drop(index=books[books.yearOfPublication=='Gallimard'].index,axis=1)

### Change the datatype of yearOfPublication to 'int'

In [0]:
#books=books.yearOfPublication.astype('int32')
books = books.astype({"yearOfPublication": 'int32'})

In [48]:
books.dtypes

ISBN                 object
bookTitle            object
bookAuthor           object
yearOfPublication     int32
publisher            object
dtype: object

### Drop NaNs in `'publisher'` column


In [0]:
books.yearOfPublication.dropna(inplace=True)

## Exploring Users dataset

In [80]:
users.dtypes

userID       int64
Location    object
Age          int32
dtype: object

In [50]:
print(users.shape)
users.head()

(278858, 3)


,userID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


### Get all unique values in ascending order for column `Age`

In [52]:
users.Age.sort_values(ascending=True).unique()

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 113., 114., 115., 116., 118., 119., 123., 124., 127.,
       128., 132., 133., 136., 137., 138., 140., 141., 143., 146., 147.,
       148., 151., 152., 156., 157., 159., 162., 168., 172., 175., 183.,
       186., 189., 199., 200., 201., 204., 207., 20

Age column has some invalid entries like nan, 0 and very high values like 100 and above

### Values below 5 and above 90 do not make much sense for our book rating case...hence replace these by NaNs

In [0]:
users.loc[(users.Age < 5) | (users.Age > 90),['Age']]=np.nan

### Replace null values in column `Age` with mean

In [0]:
users.Age.fillna(users.Age.mean(),inplace=True)

### Change the datatype of `Age` to `int`

In [0]:
users = users.astype({"Age": 'int32'})

In [74]:
print(sorted(users.Age.unique()))

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]


## Exploring the Ratings Dataset

In [81]:
ratings.dtypes

userID         int64
ISBN          object
bookRating     int64
dtype: object

### check the shape

In [75]:
ratings.shape

(1149780, 3)

In [0]:
n_users = users.shape[0]
n_books = books.shape[0]

In [0]:
ratings.head(5)

,userID,ISBN,bookRating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


### Ratings dataset should have books only which exist in our books dataset. Drop the remaining rows

In [0]:
ratings=ratings[ratings.ISBN.isin(books.ISBN)]

In [104]:
#ratings=ratings[~ratings.ISBN.isin(books.ISBN)].dropna()
ratings.shape

(1031132, 3)

### Ratings dataset should have ratings from users which exist in users dataset. Drop the remaining rows

In [0]:
ratings=ratings[ratings.userID.isin(users.userID)]

In [106]:
#ratings[~ratings.userID.isin(users.userID)].dropna()
ratings.shape

(1031132, 3)

### Consider only ratings from 1-10 and leave 0s in column `bookRating`

In [0]:
ratings=ratings[ratings.bookRating != 0]

### Find out which rating has been given highest number of times

In [109]:
ratings.bookRating.value_counts()
# 8 rating is given highest time

8     91804
10    71225
7     66401
9     60778
5     45355
6     31687
4      7617
3      5118
2      2375
1      1481
Name: bookRating, dtype: int64

### **Collaborative Filtering Based Recommendation Systems**

### For more accurate results only consider users who have rated atleast 100 books

In [0]:
#print(ratings.groupby(['userID','bookRating']).userID.count()>=100)

In [0]:
filter_ratings_users = ratings['userID'].value_counts() > 100

In [0]:
filter_ratings_users = filter_ratings_users[filter_ratings_users].index.tolist()

In [0]:
ratings = ratings[ratings['userID'].isin(filter_ratings_users)]

In [139]:
ratings.shape

(102371, 3)

### Generating ratings matrix from explicit ratings


#### Note: since NaNs cannot be handled by training algorithms, replace these by 0, which indicates absence of ratings

In [0]:
ratings=ratings.fillna(0)
ratings=ratings[ratings.bookRating != 0]

### Generate the predicted ratings using SVD with no.of singular values to be 50

In [0]:
from surprise import SVD,Dataset,Reader

In [0]:
reader = Reader(rating_scale=(1, 10))

In [0]:
data=Dataset.load_from_df(ratings,reader)

In [177]:
algo = SVD()
algo.fit(data.build_full_trainset())

In [0]:
#predt=algo.predict('uid','iid')

### Take a particular user_id

### Lets find the recommendations for user with id `2110`

#### Note: Execute the below cells to get the variables loaded

In [0]:
userID = 2110

In [0]:
user_id = 2 #2nd row in ratings matrix and predicted matrix

### Get the predicted ratings for userID `2110` and sort them in descending order

In [0]:
bookIds =ratings['ISBN'].unique();

In [0]:
userRatings = ratings.loc[ratings['userID']==2110,'ISBN']

In [0]:
diffRatings = np.setdiff1d(bookIds,userRatings)

In [0]:
testratings = [[2110,bookIds,4] for bookIds in diffRatings]

In [0]:
pred=algo.test(testratings)

In [224]:
pred[0]

Prediction(uid=2110, iid='0000913154', r_ui=4, est=8.485427770202351, details={'was_impossible': False})

In [0]:
from collections import defaultdict

In [0]:
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [0]:
top_n=get_top_n(pred,len(pred)) # print descending order

In [227]:
get_top_n(pred,10)

defaultdict(list,
            {2110: [('0446672211', 9.649439568861059),
              ('0441172717', 9.638585402153188),
              ('0618002227', 9.636899607845569),
              ('0345361792', 9.58646724213264),
              ('067172262X', 9.548931898377747),
              ('0743454529', 9.523313971840128),
              ('0385484518', 9.50786712085902),
              ('0060987561', 9.465917372719652),
              ('0671041789', 9.4343196127391),
              ('0064471101', 9.417526269735319)]})

### Create a dataframe with name `user_data` containing userID `2110` explicitly interacted books

In [0]:
bookratingsList = []
for uid, iid, true_r, est, _ in pred:
        bookratingsList.append([2110,iid, est])

In [0]:
user_data = pd.DataFrame(bookratingsList,columns=['userID','ISBN','bookRating'])

In [231]:
user_data.head()

,userID,ISBN,bookRating
0,2110,0000913154,8.485428
1,2110,0001046438,8.012689
2,2110,000104687X,8.318143
3,2110,0001047213,8.013900
4,2110,0001047973,8.236491


In [232]:
user_data.shape

(65973, 3)

### Combine the user_data and and corresponding book data(`book_data`) in a single dataframe with name `user_full_info`

In [0]:
book_data.shape

(103, 5)

In [0]:
book_data.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
246,0151008116,Life of Pi,Yann Martel,2002,Harcourt
904,015216250X,So You Want to Be a Wizard: The First Book in ...,Diane Duane,2001,Magic Carpet Books
1000,0064472779,All-American Girl,Meg Cabot,2003,HarperTrophy
1302,0345307674,Return of the Jedi (Star Wars),James Kahn,1983,Del Rey Books
1472,0671527215,Hitchhikers's Guide to the Galaxy,Douglas Adams,1984,Pocket


In [0]:
user_full_info=pd.merge(user_data,books,on='ISBN')

In [236]:
user_full_info.head()

,userID,ISBN,bookRating,bookTitle,bookAuthor,yearOfPublication,publisher
0,2110,0000913154,8.485428,The Way Things Work: An Illustrated Encycloped...,C. van Amerongen (translator),1967,Simon &amp; Schuster
1,2110,0001046438,8.012689,Liar,Stephen Fry,0,Harpercollins Uk
2,2110,000104687X,8.318143,"T.S. Eliot Reading \The Wasteland\"" and Other ...",T.S. Eliot,1993,HarperCollins Publishers
3,2110,0001047213,8.013900,The Fighting Man,Gerald Seymour,1993,HarperCollins Publishers
4,2110,0001047973,8.236491,Brave New World,Aldous Huxley,1999,Trafalgar Square Publishing


### Get top 10 recommendations for above given userID from the books not already rated by that user

In [234]:
get_top_n(pred,10)

defaultdict(list,
            {2110: [('0446672211', 9.649439568861059),
              ('0441172717', 9.638585402153188),
              ('0618002227', 9.636899607845569),
              ('0345361792', 9.58646724213264),
              ('067172262X', 9.548931898377747),
              ('0743454529', 9.523313971840128),
              ('0385484518', 9.50786712085902),
              ('0060987561', 9.465917372719652),
              ('0671041789', 9.4343196127391),
              ('0064471101', 9.417526269735319)]})